In [ ]:
import pandas as pd
import os
import numpy as np
import random
import sys
sys.path.append('/home/graduates/cyn/Cell2Map')
from Cell2Map import common
import matplotlib.pyplot as plt
import scanpy as sc
import sys

from Cell2Map import map_utils
from Cell2Map import autoencoder
from Cell2Map import utils
import anndata as ad

# Load data

Read dataset

In [642]:
scRNA_path="../data/DCIS1/sc_RNA_DCIS1.h5ad"
cell_type_path=None
st_path="../data/BRCA_10X_Visium_ST_sample1/spatial.h5ad"
coordinates_path=None
output_folder="../result/BRCA_10X_Visium_ST_sample1/DCIS1_STsample1/"
mean_cell_numbers=5

In [643]:
celltype_col='CellType'

# Read the data from the specified paths
# sc_adata: single-cell RNA-seq data
# spatial_adata: spatial transcriptomics data
# cell_type_number_eachspot_data: estimated number of each cell type in each spot
# coordinates_data: coordinates of each spot
sc_adata, spatial_adata, coordinates_data =\
        common.read_data(scRNA_path, cell_type_path, st_path, coordinates_path,celltype_col)

# Extract the cell type information from the single-cell data
# Rename the index and the 'CellType' column for clarity
cell_type=pd.DataFrame(sc_adata.obs[celltype_col])
cell_type.index = [str(idx)[5:] for idx in cell_type.index]
cell_type['CellType'] = [str(idx)[5:] for idx in cell_type[celltype_col]]

# estimate the number of cells in each spot
cell_number_to_node_assignment=common.estimate_cell_number_RNA_reads(spatial_adata.to_df().T, int(mean_cell_numbers))

In [644]:
sc_adata,spatial_adata=map_utils.process_adatas(sc_adata,spatial_adata)

/home/graduates/anaconda3/envs/celloc/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [645]:
print(sc_adata)
print(spatial_adata)

AnnData object with n_obs × n_vars = 2933 × 15858
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.MT', 'RNA_snn_res.1', 'seurat_clusters', 'RNA_snn_res.1.5', 'RNA_snn_res.2', 'cell_type', 'UMAP_1', 'UMAP_2', 'CellType'
    var: 'n_cells'
    uns: 'log1p', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'
AnnData object with n_obs × n_vars = 2518 × 15858
    obs: 'in_tissue', 'array_row', 'array_col', 'rna_count_based_density'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'spatial', 'log1p'
    obsm: 'spatial'


In [646]:
sc_adata,spatial_adata=autoencoder.embedding_feature(sc_adata,spatial_adata,k_cutoff=5)

The graph contains 20753 edges, 2933 cells.
7.075690419365837 neighbors per cell on average.
The graph contains 0 edges, 2518 cells.
0.0 neighbors per cell on average.


100%|██████████| 500/500 [00:08<00:00, 59.84it/s]


In [647]:
Deep_map=map_utils.map_cell_to_space(sc_adata,spatial_adata,
learning_rate=0.0008,
    num_epochs=8000,
    lambda_d=2.5,
    lambda_g1=3.2,
    lambda_g2=2.2,
    b_init=cell_number_to_node_assignment,
    alpha=0.55,
    lambda_mahalanobis=0.4,
    lambda_distance=0.02,
    device='cuda:0')

/home/graduates/cyn/Cell2spotGATE/Cell2spotGATE/map_optimizer.py:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.d=torch.tensor(d,device=device, dtype=torch.float32)


seed:1000
Total_loss: 15.735, gv_term: 2.318, vg_term: 1.972, density_term: 19.579, expression_term: 1.984, distance_term:1.537,
Total_loss: 15.721, gv_term: 2.329, vg_term: 1.973, density_term: 19.579, expression_term: 1.985, distance_term:1.541,
Total_loss: 15.721, gv_term: 2.329, vg_term: 1.973, density_term: 19.579, expression_term: 1.985, distance_term:1.541,
Total_loss: 15.721, gv_term: 2.329, vg_term: 1.973, density_term: 19.579, expression_term: 1.985, distance_term:1.541,
Total_loss: 15.721, gv_term: 2.329, vg_term: 1.973, density_term: 19.579, expression_term: 1.985, distance_term:1.541,
Total_loss: 15.721, gv_term: 2.329, vg_term: 1.973, density_term: 19.579, expression_term: 1.985, distance_term:1.541,
Total_loss: 15.721, gv_term: 2.329, vg_term: 1.973, density_term: 19.579, expression_term: 1.985, distance_term:1.541,
Total_loss: 15.721, gv_term: 2.329, vg_term: 1.973, density_term: 19.579, expression_term: 1.985, distance_term:1.541,
Total_loss: 15.721, gv_term: 2.329, vg

In [648]:
Deep_map

AnnData object with n_obs × n_vars = 2933 × 2518
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.MT', 'RNA_snn_res.1', 'seurat_clusters', 'RNA_snn_res.1.5', 'RNA_snn_res.2', 'cell_type', 'UMAP_1', 'UMAP_2', 'CellType'
    var: 'in_tissue', 'array_row', 'array_col', 'rna_count_based_density'

In [649]:
sc_sp_map_df = pd.DataFrame(Deep_map.X, index=sc_adata.obs_names, columns=spatial_adata.obs_names)


In [650]:
# Remove the first 5 characters from the column and index names of the DataFrame
sc_sp_map_df.columns = [str(col)[5:] for col in sc_sp_map_df.columns]
sc_sp_map_df.index = [str(idx)[5:] for idx in sc_sp_map_df.index]

# Initialize an empty dictionary to store the results
result_dict = {}

# For each column in the DataFrame, find the cell names with the highest values
# The number of cell names to find is determined by cell_number_to_node_assignment
# Store the results in the result_dict dictionary
for i, column in enumerate(sc_sp_map_df.columns.values):
    result_dict[column] = sc_sp_map_df[column].nlargest(cell_number_to_node_assignment[i]).index

# Convert the result_dict dictionary to a DataFrame
# The 'Predict' column contains the spot names, and the 'Values' column contains the cell names
# The 'Values' column is exploded so that each cell name has its own row
max_column_names = pd.DataFrame({'Predict': list(result_dict.keys()), 'Values': [list(index_obj) for index_obj in result_dict.values()]}).explode('Values')

# Convert the cell names in the 'Values' column to strings
max_column_names['Values'] = max_column_names['Values'].astype(str)

# Set the 'Values' column as the index of the DataFrame
max_column_names = max_column_names.set_index('Values')
max_column_names=max_column_names.assign(CellType='type')

# Remove rows where the index is "nan"
max_column_names = max_column_names[max_column_names.index!="nan"]

# For each row in the DataFrame, replace 'type' in the 'CellType' column with the actual cell type from the cell_type DataFrame
for i,cell_id in enumerate(max_column_names.index.values):
    max_column_names.iloc[i,-1]=cell_type.loc[cell_id][0]

# Reindex the coordinates_data DataFrame based on the 'Predict' column of the max_column_names DataFrame
# This gives the predicted coordinates for each cell
predict_coordinates=coordinates_data.reindex(max_column_names['Predict'])

# Add the predicted x and y coordinates to the max_column_names DataFrame
max_column_names['predict_x']=predict_coordinates['X'].values
max_column_names['predict_y']=predict_coordinates['Y'].values

# Print the DataFrame
print(max_column_names)

/tmp/ipykernel_3580355/2128575498.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_column_names.iloc[i,-1]=cell_type.loc[cell_id][0]
/tmp/ipykernel_3580355/2128575498.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_column_names.iloc[i,-1]=cell_type.loc[cell_id][0]
/tmp/ipykernel_3580355/2128575498.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_column_names.iloc[i,-1]=cell_type.loc[cell_id][0]
/tmp/ipykerne

                             Predict     CellType  predict_x  predict_y
Values                                                                 
GATGCTAAGTGCACTT  AAACAAGTATCTCCCA-1  Epithelial3      19934      16265
AGTAGTCCACACCTAA  AAACAAGTATCTCCCA-1  Epithelial3      19934      16265
TGTAGACAGTATGATG  AAACAAGTATCTCCCA-1  Epithelial3      19934      16265
AGGAAATCACTCCCTA  AAACAAGTATCTCCCA-1  Epithelial3      19934      16265
TCGCAGGGTGCCGGTT  AAACAAGTATCTCCCA-1  Epithelial3      19934      16265
...                              ...          ...        ...        ...
TACCGAACACAGCCTG  TTGTTTCATTAGTCTA-1         NK/T       9488      18780
TGATTTCAGGTAAAGG  TTGTTTCATTAGTCTA-1       B cell       9488      18780
CTCTCAGCATGACTAC  TTGTTTCCATACAACT-1      Myeloid       9056      14994
CATTGCCAGCTCGAAG  TTGTTTCCATACAACT-1      Myeloid       9056      14994
TTTAGTCCATGGGAAC  TTGTTTCCATACAACT-1          pDC       9056      14994

[11322 rows x 4 columns]


In [ ]:
assigned_locations_path = os.path.join(output_folder+"Cell2Map_DCIS1_with_STsample1.csv")
max_column_names.to_csv(assigned_locations_path)